# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
# limit = 5
# sum = 0
# generator = square_root_generator(limit)

# for sqrt_value in generator:
#     print(sqrt_value)
#     sum += sqrt_value

# print(f'The sum is {sum}') #8.382332347441762


limit = 13
generator = square_root_generator(limit)
for sqrt_value in generator:
    print(sqrt_value)
# 3.605551275463989

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [5]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [6]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_1",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="people_2",
										write_disposition="append")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708221395.7423074 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708221396.5688 is LOADED and contains no failed jobs


In [7]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_1 table below:")

peopel_1 = conn.sql("SELECT * FROM people_1").df()
display(peopel_1)

print("\n\n\n people_2 table below:")

peopel_2 = conn.sql("SELECT * FROM people_2").df()
display(peopel_2)

# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_2            │
└─────────────────────┘




 people_1 table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708221395.7423074,zgZdHnzzHQ5bYA
1,2,Person_2,27,City_A,1708221395.7423074,7mzqYbQMcI4JtA
2,3,Person_3,28,City_A,1708221395.7423074,g6OhOebrv7aqBA
3,4,Person_4,29,City_A,1708221395.7423074,hf/XDA/9QjBDbg
4,5,Person_5,30,City_A,1708221395.7423074,ZzYIcfNJd51cIA





 people_2 table below:


,id,name,age,city,occupation,_dlt_load_id,_dlt_id
0,3,Person_3,33,City_B,Job_3,1708221396.5688,Y70qDZ14EfzZWQ
1,4,Person_4,34,City_B,Job_4,1708221396.5688,2wzhmrJ1ZryHvg
2,5,Person_5,35,City_B,Job_5,1708221396.5688,b6CBE1hJgmZadw
3,6,Person_6,36,City_B,Job_6,1708221396.5688,iB2E9yGPdpN9vQ
4,7,Person_7,37,City_B,Job_7,1708221396.5688,eVQOvMGSejrIEQ
5,8,Person_8,38,City_B,Job_8,1708221396.5688,RoHayuZRkWG4kg


In [8]:
sum_peopel_1 = conn.sql("SELECT SUM(age) FROM people_1").df()
sum_peopel_2 = conn.sql("SELECT SUM(age) FROM people_2").df()
print(f'Total: {sum_peopel_1 + sum_peopel_2}')

Total:    sum(age)
0     353.0


In [9]:
data = [
    {'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'},
    {'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'},
    {'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'},
    {'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'},
    {'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'},
    {'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'},
    {'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'},
    {'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'},
    {'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'},
    {'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'},
    {'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
]

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_data')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(data,
										table_name="people",
										write_disposition="merge",
                    primary_key='ID')

# show the outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [10]:
peopel = conn.sql("SELECT * FROM people").df()
display(peopel)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,5,Person_5,30,City_A,1708222677.8691475,N3qQgHJC6htvEw,None
1,7,Person_7,37,City_B,1708222677.8691475,SUbE4OfEPW4G9w,Job_7
2,4,Person_4,29,City_A,1708222677.8691475,G9WTOdhqYw8xCg,None
3,2,Person_2,27,City_A,1708222677.8691475,tgVK4VFmQMQSrA,None
4,8,Person_8,38,City_B,1708222677.8691475,8pphXHmktNrzlQ,Job_8
5,1,Person_1,26,City_A,1708222677.8691475,cpZ7Pwqm8mw5hg,None
6,3,Person_3,28,City_A,1708222677.8691475,s2qUSjtKN1LIoQ,None
7,6,Person_6,36,City_B,1708222677.8691475,BcP5Vhuh+E/xLQ,Job_6


In [11]:
sum_peopel = conn.sql("SELECT SUM(age) FROM people").df()
print(f'Sum of age: {sum_peopel}')

Sum of age:    sum(age)
0     251.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

### Question 1: What is the sum of the outputs of the generator for limit = 5?
- A: 10.23433234744176
- B: 7.892332347441762
- **C: 8.382332347441762**
- D: 9.123332347441762

### Question 2: What is the 13th number yielded by the generator?
- A: 4.236551275463989
- **B: 3.605551275463989**
- C: 2.345551275463989
- D: 5.678551275463989

### Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.
- **A: 353**
- B: 365
- C: 378
- D: 390

### Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.
- **A: 215**
- B: 266
- C: 241
- D: 258

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX